# Customer Segmentation using Clustering
***
This mini-project is based on [this blog post](http://blog.yhat.com/posts/customer-segmentation-using-python.html) by yhat. Please feel free to refer to the post for additional information, and solutions.

In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

## Data

The dataset contains information on marketing newsletters/e-mail campaigns (e-mail offers sent to customers) and transaction level data from customers. The transactional data shows which offer customers responded to, and what the customer ended up buying. The data is presented as an Excel workbook containing two worksheets. Each worksheet contains a different dataset.

In [2]:
df_offers = pd.read_excel("./WineKMC.xlsx", sheet_name=0)
df_offers.columns = ["offer_id", "campaign", "varietal", "min_qty", "discount", "origin", "past_peak"]
df_offers.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


We see that the first dataset contains information about each offer such as the month it is in effect and several attributes about the wine that the offer refers to: the variety, minimum quantity, discount, country of origin and whether or not it is past peak. The second dataset in the second worksheet contains transactional data -- which offer each customer responded to.

In [3]:
df_transactions = pd.read_excel("./WineKMC.xlsx", sheet_name=1)
df_transactions.columns = ["customer_name", "offer_id"]
df_transactions['n'] = 1
df_transactions.head()

,customer_name,offer_id,n
0,Smith,2,1
1,Smith,24,1
2,Johnson,17,1
3,Johnson,24,1
4,Johnson,26,1


## Data wrangling

We're trying to learn more about how our customers behave, so we can use their behavior (whether or not they purchased something based on an offer) as a way to group similar minded customers together. We can then study those groups to look for patterns and trends which can help us formulate future offers.

The first thing we need is a way to compare customers. To do this, we're going to create a matrix that contains each customer and a 0/1 indicator for whether or not they responded to a given offer. 

<div class="span5 alert alert-info">
<h3>Checkup Exercise Set I</h3>

<p><b>Exercise:</b> Create a data frame where each row has the following columns (Use the pandas [`merge`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) and [`pivot_table`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) functions for this purpose):
<ul>
<li> customer_name
<li> One column for each offer, with a 1 if the customer responded to the offer
</ul>
<p>Make sure you also deal with any weird values such as `NaN`. Read the documentation to develop your solution.</p>
</div>

In [55]:
# merge the two datasets on offer_id, take only customer_name and offer_id columns
customers_offers = df_transactions.merge(df_offers, on='offer_id')[['customer_name', 'offer_id', 'n']]

# pivot the dataframe, filling in missing values with 0 for no response
X = pd.pivot_table(customers_offers, values='n', 
               columns='offer_id', index='customer_name',
              fill_value=0)
X.head()

offer_id,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
customer_name,,,,,,,,,,,,,,,,,,,,,
Adams,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
Allen,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
Anderson,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
Bailey,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Baker,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


## K-Means Clustering

Recall that in K-Means Clustering we want to *maximize* the distance between centroids and *minimize* the distance between data points and the respective centroid for the cluster they are in. True evaluation for unsupervised learning would require labeled data; however, we can use a variety of intuitive metrics to try to pick the number of clusters K. We will introduce two methods: the Elbow method, the Silhouette method and the gap statistic.

### Choosing K: The Elbow Sum-of-Squares Method

The first method looks at the sum-of-squares error in each cluster against $K$. We compute the distance from each data point to the center of the cluster (centroid) to which the data point was assigned. 

$$SS = \sum_k \sum_{x_i \in C_k} \sum_{x_j \in C_k} \left( x_i - x_j \right)^2 = \sum_k \sum_{x_i \in C_k} \left( x_i - \mu_k \right)^2$$

where $x_i$ is a point, $C_k$ represents cluster $k$ and $\mu_k$ is the centroid for cluster $k$. We can plot SS vs. $K$ and choose the *elbow point* in the plot as the best value for $K$. The elbow point is the point at which the plot starts descending much more slowly. 

<div class="span5 alert alert-info">
<h3>Checkup Exercise Set II</h3>

<p><b>Exercise:</b></p> 
<ul>
<li> What values of $SS$ do you believe represent better clusterings? Why?
<li> Create a numpy matrix `x_cols` with only the columns representing the offers (i.e. the 0/1 colums) 
<li> Write code that applies the [`KMeans`](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) clustering method from scikit-learn to this matrix. 
<li> Construct a plot showing $SS$ for each $K$ and pick $K$ using this plot. For simplicity, test $2 \le K \le 10$.
<li> Make a bar chart showing the number of points in each cluster for k-means under the best $K$.
<li> What challenges did you experience using the Elbow method to pick $K$?
</ul>
</div>

#### What values of  𝑆𝑆  do you believe represent better clusterings? Why?

For a given value of K, a lower SS is better because each point that belongs to a cluster is closer to the cluster's centroid. Therefore our choice of cluster centroids can describe the data well.

#### Create a numpy matrix `x_cols` with only the columns representing the offers (i.e. the 0/1 colums)

In [5]:
x_cols = np.array(X)

#### Write code that applies the [`KMeans`](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) clustering method from scikit-learn to this matrix.

In [6]:
def choose_K(data, n_clusters, random_state=None):
    """
    Creates KMeans model for each n in n_clusters. Returns
    the model, SS (inertia), and labels on each iteration.
    """
    
    results = {}
    
    for n in n_clusters:
        model = KMeans(n_clusters=n, random_state=random_state)
        model.fit(data)
        results[n] = {'model': model, 'inertia': model.inertia_, 'labels': model.labels_}
        
    return results

In [7]:
# choose range for n_clusters
n = range(2,11)

# get results for each value of n_clusters
results = choose_K(x_cols, n_clusters=n, random_state=42)

# examine results for n_clusters=2
results[2]

{'model': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=42, tol=0.0001, verbose=0),
 'inertia': 251.46031746031744,
 'labels': array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1])}

In [8]:
# create array of K and SS to plot
to_plot = np.array([n, [results[i]['inertia'] for i in results]])

#### Construct a plot showing  𝑆𝑆  for each  𝐾  and pick  𝐾  using this plot. For simplicity, test  2≤𝐾≤10 .

In [9]:
trace0 = go.Scattergl(x=to_plot[0], y=to_plot[1], mode='lines+markers')

layout = go.Layout(title='SS vs K',
                  xaxis=dict(title='K'),
                  yaxis=dict(title='SS'))

fig = go.Figure([trace0], layout)

iplot(fig, filename='SS-vs-K.html')

#### Make a bar chart showing the number of points in each cluster for k-means under the best  𝐾 .

In [10]:
to_plot_bar = np.unique(results[5]['labels'], return_counts=True)

trace0 = go.Bar(x=to_plot_bar[0], y=to_plot_bar[1])

layout = go.Layout(title='Number of Labels per K',
                  xaxis=dict(title='K'),
                  yaxis=dict(title='Label Count'))

fig = go.Figure([trace0], layout)

iplot(fig, filename='label-counts-K5.html')

#### What challenges did you experience using the Elbow method to pick  𝐾 ?

Within the range of 2 to 10 that we examined, it's hard for me to see a clear knee. I suppose this will be a challenge many times when trying to eyeball the plot for the knee. The method is both subjective and lacks concrete criteria for choosing K.

### Choosing K: The Silhouette Method

There exists another method that measures how well each datapoint $x_i$ "fits" its assigned cluster *and also* how poorly it fits into other clusters. This is a different way of looking at the same objective. Denote $a_{x_i}$ as the *average* distance from $x_i$ to all other points within its own cluster $k$. The lower the value, the better. On the other hand $b_{x_i}$ is the minimum average distance from $x_i$ to points in a different cluster, minimized over clusters. That is, compute separately for each cluster the average distance from $x_i$ to the points within that cluster, and then take the minimum. The silhouette $s(x_i)$ is defined as

$$s(x_i) = \frac{b_{x_i} - a_{x_i}}{\max{\left( a_{x_i}, b_{x_i}\right)}}$$

The silhouette score is computed on *every datapoint in every cluster*. The silhouette score ranges from -1 (a poor clustering) to +1 (a very dense clustering) with 0 denoting the situation where clusters overlap. Some criteria for the silhouette coefficient is provided in the table below.

<pre>

| Range       | Interpretation                                |
|-------------|-----------------------------------------------|
| 0.71 - 1.0  | A strong structure has been found.            |
| 0.51 - 0.7  | A reasonable structure has been found.        |
| 0.26 - 0.5  | The structure is weak and could be artificial.|
| < 0.25      | No substantial structure has been found.      |

</pre>
Source: http://www.stat.berkeley.edu/~spector/s133/Clus.html

Fortunately, scikit-learn provides a function to compute this for us (phew!) called [`sklearn.metrics.silhouette_score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html). Take a look at [this article](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html) on picking $K$ in scikit-learn, as it will help you in the next exercise set.

<div class="span5 alert alert-info">
<h3>Checkup Exercise Set III</h3>

<p><b>Exercise:</b> Using the documentation for the `silhouette_score` function above, construct a series of silhouette plots like the ones in the article linked above.</p>

<p><b>Exercise:</b> Compute the average silhouette score for each $K$ and plot it. What $K$ does the plot suggest we should choose? Does it differ from what we found using the Elbow method?</p>
</div>

#### Exercise: Using the documentation for the `silhouette_score` function above, construct a series of silhouette plots like the ones in the article linked above.

In [11]:
from sklearn.metrics import silhouette_samples, silhouette_score

def choose_K(data, n_clusters, random_state=None):
    """
    Creates KMeans model for each n in n_clusters. Returns
    the model, SS (inertia), and labels on each iteration.
    """
    
    results = {}
    
    for n in n_clusters:
        model = KMeans(n_clusters=n, random_state=random_state)
        model.fit(data)
        
        sample_silhouette_values = silhouette_samples(data, model.labels_)
        silhouette_score_ = silhouette_score(data, model.labels_)
        
        results[n] = {'model': model, 'inertia': model.inertia_, 
                      'labels': model.labels_, 'silhouette': silhouette_score_,
                      'silhuoette_samples': sample_silhouette_values}
        
    return results

In [12]:
# choose range for n_clusters
n = range(2,11)

# get results for each value of n_clusters
results = choose_K(x_cols, n_clusters=n, random_state=42)

# examine results for n_clusters=2
results[2]

{'model': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=42, tol=0.0001, verbose=0),
 'inertia': 251.46031746031744,
 'labels': array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1]),
 'silhouette': 0.09174871508750351,
 'silhuoette_samples': array([ 2.66944891e-01, -1.13071856e-02,  5.46824603e-02,  2.38239844e-01,
         8.66724857e-02,  1.07138243e-02,  8.46595901e-02,  2.60403849e-01,
        -5.39660119e-02,  2.97686782e-01, -3.88898752e-03,  6.96545446e-02,
         2.75531625e-01,  4.71135801e-02, -1.02831052e-01,  5.46824603e-02,
         1.30939315e-02,  8.46595901e-02,  2.32446224e-01, 

In [13]:
df = pd.DataFrame({'label': results[2]['labels'], 'sil': results[5]['silhuoette_samples']})
df = df.sort_values(['label', 'sil'], ascending=False).reset_index(drop=True).reset_index()
df.head()

,index,label,sil
0,0,1,0.446331
1,1,1,0.446331
2,2,1,0.439586
3,3,1,0.439586
4,4,1,0.412600


In [14]:
trace0 = go.Bar(x=df[df.label==0]['index'], y=df[df.label==0]['sil'], name='Label=0')
trace1 = go.Bar(x=df[df.label==1]['index'], y=df[df.label==1]['sil'], name='Label=1')

layout = go.Layout(title='Silhouette Coefficients by Label',
                  xaxis=dict(title='Sample Number'),
                  yaxis=dict(title='Silhouette Coefficient'))

fig = go.Figure([trace0, trace1], layout)

iplot(fig, filename='silhouettes-K5.html')

#### Exercise: Compute the average silhouette score for each  𝐾  and plot it. What  𝐾  does the plot suggest we should choose? Does it differ from what we found using the Elbow method?

In [15]:
# create array of K and SS to plot
to_plot = np.array([n, [results[i]['silhouette'] for i in results]])

trace0 = go.Scatter(x=to_plot[0], y=to_plot[1])

layout = go.Layout(title='Average Silhouette Score vs K',
                  xaxis=dict(title='K'),
                  yaxis=dict(title='Average Silhouette Score'))

fig = go.Figure([trace0], layout)

iplot(fig, filename='silhouette-scores.html')

The plot above suggests the optimal value of K is 5. This is the same K I picked using the elbow method, albeit somewhat randomly.

### Choosing $K$: The Gap Statistic

There is one last method worth covering for picking $K$, the so-called Gap statistic. The computation for the gap statistic builds on the sum-of-squares established in the Elbow method discussion, and compares it to the sum-of-squares of a "null distribution," that is, a random set of points with no clustering. The estimate for the optimal number of clusters $K$ is the value for which $\log{SS}$ falls the farthest below that of the reference distribution:

$$G_k = E_n^*\{\log SS_k\} - \log SS_k$$

In other words a good clustering yields a much larger difference between the reference distribution and the clustered data. The reference distribution is a Monte Carlo (randomization) procedure that constructs $B$ random distributions of points within the bounding box (limits) of the original data and then applies K-means to this synthetic distribution of data points.. $E_n^*\{\log SS_k\}$ is just the average $SS_k$ over all $B$ replicates. We then compute the standard deviation $\sigma_{SS}$ of the values of $SS_k$ computed from the $B$ replicates of the reference distribution and compute

$$s_k = \sqrt{1+1/B}\sigma_{SS}$$

Finally, we choose $K=k$ such that $G_k \geq G_{k+1} - s_{k+1}$.

### Aside: Choosing $K$ when we Have Labels

Unsupervised learning expects that we do not have the labels. In some situations, we may wish to cluster data that is labeled. Computing the optimal number of clusters is much easier if we have access to labels. There are several methods available. We will not go into the math or details since it is rare to have access to the labels, but we provide the names and references of these measures.

* Adjusted Rand Index
* Mutual Information
* V-Measure
* Fowlkes–Mallows index

See [this article](http://scikit-learn.org/stable/modules/clustering.html) for more information about these metrics.

## Visualizing Clusters using PCA

How do we visualize clusters? If we only had two features, we could likely plot the data as is. But we have 100 data points each containing 32 features (dimensions). Principal Component Analysis (PCA) will help us reduce the dimensionality of our data from 32 to something lower. For a visualization on the coordinate plane, we will use 2 dimensions. In this exercise, we're going to use it to transform our multi-dimensional dataset into a 2 dimensional dataset.

This is only one use of PCA for dimension reduction. We can also use PCA when we want to perform regression but we have a set of highly correlated variables. PCA untangles these correlations into a smaller number of features/predictors all of which are orthogonal (not correlated). PCA is also used to reduce a large set of variables into a much smaller one.

<div class="span5 alert alert-info">
<h3>Checkup Exercise Set IV</h3>

<p><b>Exercise:</b> Use PCA to plot your clusters:</p>

<ul>
<li> Use scikit-learn's [`PCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) function to reduce the dimensionality of your clustering data to 2 components
<li> Create a data frame with the following fields:
  <ul>
  <li> customer name
  <li> cluster id the customer belongs to
  <li> the two PCA components (label them `x` and `y`)
  </ul>
<li> Plot a scatterplot of the `x` vs `y` columns
<li> Color-code points differently based on cluster ID
<li> How do the clusters look? 
<li> Based on what you see, what seems to be the best value for $K$? Moreover, which method of choosing $K$ seems to have produced the optimal result visually?
</ul>

<p><b>Exercise:</b> Now look at both the original raw data about the offers and transactions and look at the fitted clusters. Tell a story about the clusters in context of the original data. For example, do the clusters correspond to wine variants or something else interesting?</p>
</div>

#### Exercise: Use PCA to plot your clusters:

In [16]:
X.head()

offer_id,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
customer_name,,,,,,,,,,,,,,,,,,,,,
Adams,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
Allen,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
Anderson,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
Bailey,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Baker,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [17]:
results[5]

{'model': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=42, tol=0.0001, verbose=0),
 'inertia': 203.76651387827857,
 'labels': array([1, 0, 4, 1, 0, 2, 4, 1, 2, 1, 2, 4, 1, 0, 1, 4, 0, 4, 1, 2, 1, 1,
        2, 2, 4, 3, 3, 0, 0, 4, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 4, 4, 1, 0,
        1, 2, 1, 1, 0, 0, 0, 3, 4, 4, 2, 1, 4, 1, 1, 0, 0, 1, 0, 1, 1, 4,
        4, 0, 2, 0, 3, 0, 2, 1, 0, 1, 4, 2, 1, 4, 3, 0, 3, 4, 1, 1, 1, 3,
        0, 1, 0, 1, 0, 1, 3, 2, 1, 3, 0, 2]),
 'silhouette': 0.14531568820332333,
 'silhuoette_samples': array([ 0.27593965,  0.02089286,  0.4126    ,  0.22380855, -0.05294596,
         0.05211971,  0.44633143,  0.23644687,  0.03864241,  0.29553117,
         0.02741177,  0.40696378,  0.27355487,  0.10027515,  0.1095269 ,
         0.4126    ,  0.05553385,  0.44633143,  0.23410042,  0.02883248,
         0.3372165 ,  0.10201212,  0.04025468,  0.08324257,  0.32681001

In [18]:
import sklearn.decomposition

pca = sklearn.decomposition.PCA(n_components=2)
X_reduced = pca.fit_transform(X)

reduced = pd.DataFrame(X_reduced, columns=['x', 'y'], index=X.index).reset_index()
reduced['label'] = results[5]['labels']
reduced.head()

,customer_name,x,y,label
0,Adams,1.007580,0.108215,1
1,Allen,-0.287539,0.044715,0
2,Anderson,-0.392032,1.038391,4
3,Bailey,0.699477,-0.022542,1
4,Baker,0.088183,-0.471695,0


In [19]:
def plot_pca(X, K_range):
    
    # for each value of K
    for K in K_range:
        # fit data
        pca = sklearn.decomposition.PCA(n_components=2)
        X_reduced = pca.fit_transform(X)

        reduced = pd.DataFrame(X_reduced, columns=['x', 'y'], index=X.index).reset_index()
        reduced['label'] = results[K]['labels']
        
        traces=[]
        
        for l in np.sort(reduced.label.unique()):
        
            traces.append(go.Scattergl(x=reduced[reduced.label==l]['x'], 
                                       y=reduced[reduced.label==l]['y'], 
                                       mode='markers', name='Cluster %s' % l))
            
        layout = go.Layout(title='PCA Components K=%s' % K,
                  xaxis=dict(title='Principal Component 1'),
                  yaxis=dict(title='Principal Component 2'))

        fig = go.Figure(traces, layout)

        iplot(fig, filename='PCA%s.html' % K)

In [20]:
plot_pca(X, range(2,11))

Because this method is visual similar to the elbow method, it's subjective. From these plots I would say K=4 looks like the best choice for K. Below K=4, the data seems underfit and above it seems overfit.

#### Exercise: Now look at both the original raw data about the offers and transactions and look at the fitted clusters. Tell a story about the clusters in context of the original data. For example, do the clusters correspond to wine variants or something else interesting?

To answer this question, we'll use the model for K=5.

In [21]:
results[5]

{'model': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=42, tol=0.0001, verbose=0),
 'inertia': 203.76651387827857,
 'labels': array([1, 0, 4, 1, 0, 2, 4, 1, 2, 1, 2, 4, 1, 0, 1, 4, 0, 4, 1, 2, 1, 1,
        2, 2, 4, 3, 3, 0, 0, 4, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 4, 4, 1, 0,
        1, 2, 1, 1, 0, 0, 0, 3, 4, 4, 2, 1, 4, 1, 1, 0, 0, 1, 0, 1, 1, 4,
        4, 0, 2, 0, 3, 0, 2, 1, 0, 1, 4, 2, 1, 4, 3, 0, 3, 4, 1, 1, 1, 3,
        0, 1, 0, 1, 0, 1, 3, 2, 1, 3, 0, 2]),
 'silhouette': 0.14531568820332333,
 'silhuoette_samples': array([ 0.27593965,  0.02089286,  0.4126    ,  0.22380855, -0.05294596,
         0.05211971,  0.44633143,  0.23644687,  0.03864241,  0.29553117,
         0.02741177,  0.40696378,  0.27355487,  0.10027515,  0.1095269 ,
         0.4126    ,  0.05553385,  0.44633143,  0.23410042,  0.02883248,
         0.3372165 ,  0.10201212,  0.04025468,  0.08324257,  0.32681001

In [24]:
groups = X.copy()
groups['label'] = results[5]['labels']
groups.head()

offer_id,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,label
customer_name,,,,,,,,,,,,,,,,,,,,,
Adams,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
Allen,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
Anderson,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,4
Bailey,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
Baker,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0


To get an idea of the composition of each cluster, I will sum the number of responses from customers for each offer, grouped by clusters. Then get the top 5 (most responses) offers for each group.

In [28]:
# subset the data for each label and compute
# the number of responses to each offer
# take the top 5 offers with the most responses
zero = groups[groups.label==0].sum()[:-1].nlargest(5)
one = groups[groups.label==1].sum()[:-1].nlargest(5)
two = groups[groups.label==2].sum()[:-1].nlargest(5)
three = groups[groups.label==3].sum()[:-1].nlargest(5)
four = groups[groups.label==4].sum()[:-1].nlargest(5)

The result for cluster 0 for example has 9 customers that responded to offer 31, and 7 that responded to offers 4 and 9.

In [43]:
one

offer_id
30    17
8     16
29    16
7     15
18    13
dtype: int64

Now I'll get the offer meta data for each group and display them below.

In [32]:
# slice the offers data with the indices from the top 5
# offers found above. 
cluster0 = df_offers.set_index('offer_id').loc[zero.index,:]
cluster1 = df_offers.set_index('offer_id').loc[one.index,:]
cluster2 = df_offers.set_index('offer_id').loc[two.index,:]
cluster3 = df_offers.set_index('offer_id').loc[three.index,:]
cluster4 = df_offers.set_index('offer_id').loc[four.index,:]

In [33]:
cluster0

,campaign,varietal,min_qty,discount,origin,past_peak
offer_id,,,,,,
31,December,Champagne,72,89,France,False
4,February,Champagne,72,48,France,True
9,April,Chardonnay,144,57,Chile,False
11,May,Champagne,72,85,France,False
20,August,Cabernet Sauvignon,72,82,Italy,False


In [34]:
cluster1

,campaign,varietal,min_qty,discount,origin,past_peak
offer_id,,,,,,
30,December,Malbec,6,54,France,False
8,March,Espumante,6,45,South Africa,False
29,November,Pinot Grigio,6,87,France,False
7,March,Prosecco,6,40,Australia,True
18,July,Espumante,6,50,Oregon,False


In [35]:
cluster2

,campaign,varietal,min_qty,discount,origin,past_peak
offer_id,,,,,,
22,August,Champagne,72,63,France,False
31,December,Champagne,72,89,France,False
6,March,Prosecco,144,86,Chile,False
11,May,Champagne,72,85,France,False
1,January,Malbec,72,56,France,False


In [36]:
cluster3

,campaign,varietal,min_qty,discount,origin,past_peak
offer_id,,,,,,
14,June,Merlot,72,64,Chile,False
22,August,Champagne,72,63,France,False
1,January,Malbec,72,56,France,False
15,June,Cabernet Sauvignon,144,19,Italy,False
23,September,Chardonnay,144,39,South Africa,False


In [37]:
cluster4

,campaign,varietal,min_qty,discount,origin,past_peak
offer_id,,,,,,
24,September,Pinot Noir,6,34,Italy,False
26,October,Pinot Noir,144,83,Australia,False
2,January,Pinot Noir,72,17,France,False
17,July,Pinot Noir,12,47,Germany,False
1,January,Malbec,72,56,France,False


I'll also print the total number of responses within each cluster:

In [50]:
print('Total # of responses in cluster 0: %s' % zero.sum())
print('Total # of responses in cluster 1: %s' % one.sum())
print('Total # of responses in cluster 2: %s' % two.sum())
print('Total # of responses in cluster 3: %s' % three.sum())
print('Total # of responses in cluster 4: %s' % four.sum())

Total # of responses in cluster 0: 34
Total # of responses in cluster 1: 77
Total # of responses in cluster 2: 37
Total # of responses in cluster 3: 22
Total # of responses in cluster 4: 40


There are a few interesting patterns to note. First, cluster 1 contains exclusively offers with a min_qty of 6. Looking at the total number of responses printed above, we can see that this cluster has by far the highest number of total responses. This may be because the offers are much cheaper with only a minimum purchase requirement of 6 bottles. Not surprisingly, cluster 3 contains two offers with the highest number of minimum purchase quantities of 144 and has the fewest number of total responses.

Cluster 4 is also quite distinct with 4 out of the 5 wines being Pinot Noirs, while none of the other offers contain any Pinot Noirs. If we go back to the PCA scatter plots for K=5 (corresponding to the clusters we're currently looking at), it's very easy to see that clusters 1 and 4 are by far the most distinct clusters, having very little overlap with the other 3 groups. This agrees well with what we see when we qualitatively compare the offers in these groups.

Clusters 0 and 2 both have 3 champagne offers. They also both have an offer that can be a lighter sparkling wine (chardonnay and prosecco). And their 5th is a red. The offers are also mostly from France. These clusters seem very similar and could logically be combined into one group.

Cluster 3 seems to be the least distinct group. The most distinguishing feature seems to be that the campaigns were ran mostly in the summer/early fall months.

---

What we've done is we've taken those columns of 0/1 indicator variables, and we've transformed them into a 2-D dataset. We took one column and arbitrarily called it `x` and then called the other `y`. Now we can throw each point into a scatterplot. We color coded each point based on it's cluster so it's easier to see them.

<div class="span5 alert alert-info">
<h3>Exercise Set V</h3>

<p>As we saw earlier, PCA has a lot of other uses. Since we wanted to visualize our data in 2 dimensions, restricted the number of dimensions to 2 in PCA. But what is the true optimal number of dimensions?</p>

<p><b>Exercise:</b> Using a new PCA object shown in the next cell, plot the `explained_variance_` field and look for the elbow point, the point where the curve's rate of descent seems to slow sharply. This value is one possible value for the optimal number of dimensions. What is it?</p>
</div>

In [67]:
# Initialize a new PCA model with a default number of components.
import sklearn.decomposition
pca = sklearn.decomposition.PCA()
pca.fit(X)

x = list(range(1, len(pca.explained_variance_)))
y = pca.explained_variance_
y

array([0.4096489 , 0.30753551, 0.2022926 , 0.16703717, 0.15015248,
       0.1434373 , 0.13818887, 0.12192294, 0.11636172, 0.10804271,
       0.09937813, 0.09495961, 0.08690352, 0.07256738, 0.0660996 ,
       0.06245473, 0.05634388, 0.05327395, 0.04728801, 0.04393911,
       0.03900424, 0.03625783, 0.03455714, 0.03235091, 0.02940632,
       0.02618221, 0.02308167, 0.02142632, 0.018814  , 0.0165252 ,
       0.01426187, 0.0077789 ])

In [75]:
trace0 = go.Scatter(x=x, y=y, mode='lines+markers')

layout = go.Layout(title='Explained Variance vs # of Dimensions',
                  xaxis=dict(title='# of Dimensions'),
                  yaxis=dict(title='Explained Variance'))

fig = go.Figure([trace0], layout)

iplot(fig, filename='explained-var_vs_N-dimensions.html')

I would choose 5 as the optimal number of dimensions. From 1 to 5, the slope is significantly changing. Above 5, there seems to be a more consistent, monotonic, and gradual decrease in slope.

## Other Clustering Algorithms

k-means is only one of a ton of clustering algorithms. Below is a brief description of several clustering algorithms, and the table provides references to the other clustering algorithms in scikit-learn. 

* **Affinity Propagation** does not require the number of clusters $K$ to be known in advance! AP uses a "message passing" paradigm to cluster points based on their similarity. 

* **Spectral Clustering** uses the eigenvalues of a similarity matrix to reduce the dimensionality of the data before clustering in a lower dimensional space. This is tangentially similar to what we did to visualize k-means clusters using PCA. The number of clusters must be known a priori.

* **Ward's Method** applies to hierarchical clustering. Hierarchical clustering algorithms take a set of data and successively divide the observations into more and more clusters at each layer of the hierarchy. Ward's method is used to determine when two clusters in the hierarchy should be combined into one. It is basically an extension of hierarchical clustering. Hierarchical clustering is *divisive*, that is, all observations are part of the same cluster at first, and at each successive iteration, the clusters are made smaller and smaller. With hierarchical clustering, a hierarchy is constructed, and there is not really the concept of "number of clusters." The number of clusters simply determines how low or how high in the hierarchy we reference and can be determined empirically or by looking at the [dendogram](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.cluster.hierarchy.dendrogram.html).

* **Agglomerative Clustering** is similar to hierarchical clustering but but is not divisive, it is *agglomerative*. That is, every observation is placed into its own cluster and at each iteration or level or the hierarchy, observations are merged into fewer and fewer clusters until convergence. Similar to hierarchical clustering, the constructed hierarchy contains all possible numbers of clusters and it is up to the analyst to pick the number by reviewing statistics or the dendogram.

* **DBSCAN** is based on point density rather than distance. It groups together points with many nearby neighbors. DBSCAN is one of the most cited algorithms in the literature. It does not require knowing the number of clusters a priori, but does require specifying the neighborhood size.

### Clustering Algorithms in Scikit-learn
<table border="1">
<colgroup>
<col width="15%" />
<col width="16%" />
<col width="20%" />
<col width="27%" />
<col width="22%" />
</colgroup>
<thead valign="bottom">
<tr><th>Method name</th>
<th>Parameters</th>
<th>Scalability</th>
<th>Use Case</th>
<th>Geometry (metric used)</th>
</tr>
</thead>
<tbody valign="top">
<tr><td>K-Means</span></a></td>
<td>number of clusters</td>
<td>Very large<span class="pre">n_samples</span>, medium <span class="pre">n_clusters</span> with
MiniBatch code</td>
<td>General-purpose, even cluster size, flat geometry, not too many clusters</td>
<td>Distances between points</td>
</tr>
<tr><td>Affinity propagation</td>
<td>damping, sample preference</td>
<td>Not scalable with n_samples</td>
<td>Many clusters, uneven cluster size, non-flat geometry</td>
<td>Graph distance (e.g. nearest-neighbor graph)</td>
</tr>
<tr><td>Mean-shift</td>
<td>bandwidth</td>
<td>Not scalable with <span class="pre">n_samples</span></td>
<td>Many clusters, uneven cluster size, non-flat geometry</td>
<td>Distances between points</td>
</tr>
<tr><td>Spectral clustering</td>
<td>number of clusters</td>
<td>Medium <span class="pre">n_samples</span>, small <span class="pre">n_clusters</span></td>
<td>Few clusters, even cluster size, non-flat geometry</td>
<td>Graph distance (e.g. nearest-neighbor graph)</td>
</tr>
<tr><td>Ward hierarchical clustering</td>
<td>number of clusters</td>
<td>Large <span class="pre">n_samples</span> and <span class="pre">n_clusters</span></td>
<td>Many clusters, possibly connectivity constraints</td>
<td>Distances between points</td>
</tr>
<tr><td>Agglomerative clustering</td>
<td>number of clusters, linkage type, distance</td>
<td>Large <span class="pre">n_samples</span> and <span class="pre">n_clusters</span></td>
<td>Many clusters, possibly connectivity constraints, non Euclidean
distances</td>
<td>Any pairwise distance</td>
</tr>
<tr><td>DBSCAN</td>
<td>neighborhood size</td>
<td>Very large <span class="pre">n_samples</span>, medium <span class="pre">n_clusters</span></td>
<td>Non-flat geometry, uneven cluster sizes</td>
<td>Distances between nearest points</td>
</tr>
<tr><td>Gaussian mixtures</td>
<td>many</td>
<td>Not scalable</td>
<td>Flat geometry, good for density estimation</td>
<td>Mahalanobis distances to  centers</td>
</tr>
<tr><td>Birch</td>
<td>branching factor, threshold, optional global clusterer.</td>
<td>Large <span class="pre">n_clusters</span> and <span class="pre">n_samples</span></td>
<td>Large dataset, outlier removal, data reduction.</td>
<td>Euclidean distance between points</td>
</tr>
</tbody>
</table>
Source: http://scikit-learn.org/stable/modules/clustering.html

<div class="span5 alert alert-info">
<h3>Exercise Set VI</h3>

<p><b>Exercise:</b> Try clustering using the following algorithms. </p>
<ol>
<li>Affinity propagation
<li>Spectral clustering
<li>Agglomerative clustering
<li>DBSCAN
</ol>
<p>How do their results compare? Which performs the best? Tell a story why you think it performs the best.</p>
</div>


In [77]:
from sklearn.cluster import AffinityPropagation, SpectralClustering, AgglomerativeClustering, DBSCAN

In [80]:
# create the cluster models
affinity = AffinityPropagation()
spectral = SpectralClustering()
agglomerative = AgglomerativeClustering()
dbscan = DBSCAN()

I will evaluate the performance of the models in the following way. Each model will be created with default parameters. The models will all be trained on the full data set and then we will see how many clusters each model finds. We already did an extensive study of the KMeans clustering algorithm above and found the optimal number of clusters to be between K=4 and K=5. Therefore, the model that produces a number of clusters closest to between 4 and 5, will be taken to be the best out-of-box model for this data.

In [83]:
affinity.fit(X)
np.unique(affinity.labels_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
      dtype=int64)

In [84]:
spectral.fit(X)
np.unique(spectral.labels_)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [85]:
agglomerative.fit(X)
np.unique(agglomerative.labels_)

array([0, 1], dtype=int64)

In [86]:
dbscan.fit(X)
np.unique(dbscan.labels_)

array([-1], dtype=int64)

The affinity model seems to completely overfit the data with 14 clusters found! On the otherhand, the agglomerative model appears to underfit the data, producing only 2 clusters. The DBSCAN model returned a value of -1 for the labeling. This happens when the data is determined to be "too noisy" to fit. Finally, the spectral model found 8 clusters and is therefore the best model.